# PyTorch - homework 2: neural networks

-- Prof. Dorien Herremans

Please run the whole notebook with your code and submit the `.ipynb` file on eDimension that includes your answers [so after you run it]. 

In [2]:
from termcolor import colored

student_number="1002949"
student_name="Hong Pengfei"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Hong Pengfei, number: 1002949


 ## Question 1 -- XOR neural network [3pts]

a) Train an (at least) 2-layer neural network that can solve the XOR problem. 

b) Check the predictions resulting from your model in the second code box below.


In [7]:
import torch
import numpy as np

In [77]:
# load your data
X = torch.Tensor([[0,0],[0,1], [1,0], [1,1]])
Y = torch.Tensor([0,1,1,0]).view(-1,1)

# name your model xor
def xor():
    return torch.nn.Sequential(torch.nn.Linear(2,2), torch.nn.Sigmoid(), torch.nn.Linear(2,1), torch.nn.Sigmoid())
xor = xor()
# define your model loss function, optimizer, etc. 
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.SGD(xor.parameters(), lr=3e0)
# train the model
for i in range(5000):
    loss = loss_fn(xor(X), Y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [79]:
# test your model using the following functions (make sure the output is printed and saved when you submit this notebook):
# depending on how you defined your network you may need to slightly tweek the below prediction function

test = [[0,0],[0,1],[1,1],[1,0]]

for trial in test: 
    Xtest = torch.Tensor(trial)
    y_hat = xor(Xtest)
    print("{0} xor {1} = {2}".format(int(Xtest[0]), int(Xtest[1]), 1 if y_hat > 0.5 else 0))


0 xor 0 = 0
0 xor 1 = 1
1 xor 1 = 0
1 xor 0 = 1


## Question 2  [2pts]

Imagine a neural network model for a multilabel classification task. 

a) Which loss function should you use?

b) The resulting trained modal has a high variance error. Give 4 possible solutions to improve the model. 


* answer A
  - I would use `BCEWithLogitsLoss` without any activation or `BCELoss` with `sigmoid` activation.
* answer B
  - 1 Add regularisation
  - 2 Features selection to decrease the number of features.
  - 3 early stopping
  - 4 decrease the model size



## Question 3 - Improve hit classification [5pts]

Remember the hit predicton dataset from last week? 

a) Improve the model using a multiplayer perceptron. 

b) Make sure to run your models on the GPU. 

c) Tweek the hyperparameters such as number of nodes or layers, or other. Show two possible configurations and explain which works better and very briefly explain why this may be the case. 




In [105]:
# code your model 1
# Your code here
import torch
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from fastprogress import progress_bar
# load data
ds_trn = pd.read_csv("https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv")
ds_trn, ds_val = train_test_split(ds_trn.values, test_size=0.3)
x_trn, y_trn = torch.Tensor(ds_trn[:,:-1]).cuda(), torch.Tensor(ds_trn[:,-1]).cuda()
x_val, y_val = torch.Tensor(ds_val[:,:-1]).cuda(), torch.Tensor(ds_val[:,-1]).cuda()

# define logistic regression model
hid_dim = input_dim = len(x_trn[0])
model1 = torch.nn.Sequential(torch.nn.Linear(input_dim, hid_dim*8), torch.nn.Sigmoid(), torch.nn.Linear(hid_dim*8, 1), torch.nn.Sigmoid()).cuda()
loss_fn = torch.nn.BCELoss(); optimizer = torch.optim.Adam(model1.parameters(), lr=5e-5)
# train model


def train(model):
    epoch = 1000
    best_loss = np.inf
    patient = 0
    for i in progress_bar(range(epoch)):
        y_pred = model(x_trn)
        loss = loss_fn(y_pred, y_trn.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            val_loss = torch.nn.BCELoss()(model(x_val), y_val.unsqueeze(1))
            print(f"epoch: {i} \t train_loss: {loss.data} \t val_loss: {val_loss}")
            if val_loss < best_loss:
                best_loss = val_loss
                from copy import deepcopy
                best_params = deepcopy(model.state_dict())
            else:
                patient += 1
            if patient > 2: print("Early Stopping"); break
    return best_params

best_params = train(model1)
model1.load_state_dict(best_params)

epoch: 0 	 train_loss: 0.6721823811531067 	 val_loss: 0.6831036806106567
epoch: 100 	 train_loss: 0.6176725625991821 	 val_loss: 0.6781402826309204
epoch: 200 	 train_loss: 0.5978724360466003 	 val_loss: 0.6738325953483582
epoch: 300 	 train_loss: 0.5811390280723572 	 val_loss: 0.6720290780067444
epoch: 400 	 train_loss: 0.5660857558250427 	 val_loss: 0.671325147151947
epoch: 500 	 train_loss: 0.551995575428009 	 val_loss: 0.6713421940803528
epoch: 600 	 train_loss: 0.538661003112793 	 val_loss: 0.6722189784049988
epoch: 700 	 train_loss: 0.5261579751968384 	 val_loss: 0.6743215322494507
Early Stopping


<All keys matched successfully>

In [106]:
# evaluate model 1 (called model1 here)


import pandas as pd 

def run_evaluation(my_model):

  test = pd.read_csv('https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv')
  labels = test.iloc[:,-1]
  test = test.drop('Topclass1030', axis=1)
  testdata = torch.Tensor(test.values)
  testlabels = torch.Tensor(labels.values).view(-1,1)

  TP = 0
  TN = 0
  FN = 0
  FP = 0

  for i in range(0, testdata.size()[0]): 
    # print(testdata[i].size())
    Xtest = torch.Tensor(testdata[i])
    y_hat = my_model(Xtest)
    
    if y_hat > 0.5:
      prediction = 1
    else: 
      prediction = 0

    if (prediction == testlabels[i]):
      if (prediction == 1):
        TP += 1
      else: 
        TN += 1

    else:
      if (prediction == 1):
        FP += 1
      else: 
        FN += 1

  print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
  print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
  rate = TP/(FN+TP)
  print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

run_evaluation(model1.cpu())

True Positives: 46, True Negatives: 10
False Positives: 19, False Negatives: 4
Class specific accuracy of correctly predicting a hit song is 0.92


In [99]:
# code your model 2
hid_dim = input_dim = len(x_trn[0])
model2 = torch.nn.Sequential(torch.nn.Linear(input_dim, hid_dim//4), torch.nn.Sigmoid(), 
                             torch.nn.Linear(hid_dim//4, hid_dim//16), torch.nn.Sigmoid(), 
                             torch.nn.Linear(hid_dim//16, 1), torch.nn.Sigmoid()).cuda()
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-5)
best_params = train(model2)
model2.load_state_dict(best_params)

epoch: 0 	 train_loss: 0.6782657504081726 	 val_loss: 0.6830390691757202
epoch: 100 	 train_loss: 0.6776914596557617 	 val_loss: 0.6826098561286926
epoch: 200 	 train_loss: 0.6771224141120911 	 val_loss: 0.6821855902671814
epoch: 300 	 train_loss: 0.6765583753585815 	 val_loss: 0.6817662119865417
epoch: 400 	 train_loss: 0.6759992241859436 	 val_loss: 0.6813516020774841
epoch: 500 	 train_loss: 0.675445020198822 	 val_loss: 0.6809414625167847
epoch: 600 	 train_loss: 0.674895703792572 	 val_loss: 0.6805360913276672
epoch: 700 	 train_loss: 0.6743513345718384 	 val_loss: 0.6801351308822632
epoch: 800 	 train_loss: 0.6738115549087524 	 val_loss: 0.6797382831573486
epoch: 900 	 train_loss: 0.6732765436172485 	 val_loss: 0.6793458461761475


<All keys matched successfully>

In [100]:
# evaluate model 2 (called model2 here)
run_evaluation(model2.cpu())

True Positives: 50, True Negatives: 0
False Positives: 29, False Negatives: 0
Class specific accuracy of correctly predicting a hit song is 1.0


Which works better and why do you think this may be (very briefly)? 


**[your answer here, also please summarise the differences between your two models]**
- The first model is a 2 layered flat model with 8 times bigger hidden units, so the model is likely to overfit to the dataset since it has so many hidden units.
- The second model is a 3 layered deep model with decreasing 4 times hidden units, so the model is likely to learn the high level interactions between music features while not overfitting too much, that's probably why it have better performance.

Additionally, submit your results [here](https://forms.gle/NtJJEE7Wm5ZRM3Je7) for 'Class specific accuracy of correctly predicting a hit song' and see if you got the best performance of the class! Good luck!